## Introduction to Neural Inverse Design of Nanostructures (NIDN)

### TODO

In [ ]:
### Imports (TODO remove this when finished)
%load_ext autoreload
%autoreload 2

# Append root folder in case you haven't installed NIDN
import sys
sys.path.append("../")

import nidn

## Choosing your design target

### TODO (some filter spectrum?

### Reference to Running_TRCWA notebook

### Mention of nidn.get_frequency_points(cfg) to get target frequencies

In [ ]:
cfg = nidn.load_default_cfg() # Load default cfg as starting point

cfg.N_freq = 20               # Let's investigate 20 frequency points
#OSR
# cfg.target_reflectance_spectrum =   10*[0.0] + 10*[1.0]
# cfg.target_transmittance_spectrum = 10*[0.0] + 10*[0.0]
# cfg.physical_wavelength_range[0] = 2.5e-7
# cfg.physical_wavelength_range[1] = 3e-5
# cfg.freq_distribution = "log"

# 1550 Filter
cfg.target_reflectance_spectrum =   12*[1.0] + [0.0] + 7*[1.0]
cfg.target_transmittance_spectrum = 12*[0.0] + [1.0] + 7*[0.0]
cfg.physical_wavelength_range[0] = 1e-6
cfg.physical_wavelength_range[1] = 1e-5
cfg.freq_distribution = "linear"

cfg.L = 1.

nidn.plot_spectrum(cfg,
                   cfg.target_reflectance_spectrum,
                   cfg.target_transmittance_spectrum)

physical_freqs, normalized_freqs = nidn.get_frequency_points(cfg)
print("Physical frequencies are:")
print(physical_freqs)

## Core settings in NIDN and available layers

TODO: Link to doc page where all config parameters are explained.

### TRCWA and its layers

#### Uniform Layers

#### Heterogeneous Layers

### Neural Inverse Design

#### Our inverse design setup maybe figure from website?

#### Direct epsilon estimation (regression)

#### Utilizing real materials (classification)
TODO: Mention problem of differentiability of argmax

TODO: Material collection

### Example 1 - Five uniform layers with unbound epsilon

In [ ]:
cfg.Nx = 9 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 9
cfg.eps_oversampling = 3
cfg.learning_rate = 8e-5
cfg.N_layers = 3 # Choose number of layers
cfg.type = "classification" # Choose type as described above
cfg.iterations = 3000 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
#Show all used settings
nidn.print_cfg(cfg)

In [ ]:
nidn.run_training(cfg);

### Result interpretation
#### Loss plot
TODO Explain

In [ ]:
nidn.plot_losses(cfg)

#### Spectra plot
TODO Explain

In [ ]:
nidn.plot_spectra(cfg)

#### Absolute grid values plot
TODO Explain
TODO Explain layers

In [ ]:
nidn.plot_model_grid(cfg)

#### Epsilon vs frequency and real materials
TODO Explain

In [ ]:
# nidn.plot_eps_per_point(cfg,"titanium_oxide")
nidn.plot_eps_per_point(cfg)

In [ ]:
eps,_ = nidn.model_to_eps_grid(cfg.model,cfg)
R,T = nidn.compute_spectrum(eps,cfg)

In [ ]:
import torch
absorp_spectrum = torch.tensor(1.) - torch.tensor(R) - torch.tensor(T)

In [ ]:
absorp_spectrum

In [ ]:
absorption_of_sun = torch.mean(absorp_spectrum[:10])
emiss_IR = torch.mean(absorp_spectrum[10:])
print("emiss_IR = ",emiss_IR.item())
print("absorption_of_sun = ",absorption_of_sun.item())
print("absorption_of_sun / emiss_IR =",(absorption_of_sun / emiss_IR).item())

### Example 2 - 2 patterned layers with unbound epsilon

TODO

In [ ]:
cfg.pop("model",None); # Forget the old model
cfg.Nx = 16 # Set layer size  to 16x16 (each of the grid points has its own epsilon now)
cfg.Ny = 16
cfg.eps_oversampling = 4
cfg.N_layers = 2 # Less layer to keep compute managable
cfg.type = "regression" # Choose type as described above (for now still regression)
cfg.iterations = 250 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

In [ ]:
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg)

### Example 3 - Patterned layer with bound epsilon

TODO

In [ ]:
cfg.pop("model",None); # Forget the old model
cfg.Nx = 16 # Set layer size  to 16x16 (each of the grid points has its own epsilon now)
cfg.Ny = 16
cfg.eps_oversampling = 8
cfg.N_layers = 2 # Less layer to keep compute managable
cfg.type = "classification" # Choose type as described above (for now still regression)
cfg.iterations = 250 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

#### Material ID plot
TODO

In [ ]:
nidn.plot_material_grid(cfg)

In [ ]:
# The other plots
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg)

In [ ]:
# In case you want to save results you can use this handy function
# to save it to the results folder with a current timestamp
nidn.save_run(cfg)

# You can save all available plots to a single folder using this function
nidn.save_all_plots(cfg,save_path="/results/example/")